# Kaggle Playground Series S4E9 Competition

Walter Reade, Ashley Chow. (2024). Regression of Used Car Prices. Kaggle. https://kaggle.com/competitions/playground-series-s4e9

## IMPORTS AND SETUP

In [1]:
# Imports:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Ignore harmless warnings
warnings.filterwarnings('ignore')

# Plot rcParams:
plt.rcParams['figure.figsize'] = (15, 7)
plt.rcParams['figure.dpi'] = 120
plt.rcParams['figure.titlesize'] = 22
plt.rcParams['figure.titleweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 22
plt.rcParams['axes.titleweight'] = 'bold'
plt.style.use('ggplot')

## DATA READ

In [2]:
# Read train df:
df = pd.read_csv(filepath_or_buffer='train.csv')
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [3]:
# Df columns:
df.columns

Index(['id', 'brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'clean_title',
       'price'],
      dtype='object')

In [4]:
# Read test data:
test_df = pd.read_csv('test.csv')
test_df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,188533,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes
1,188534,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes
2,188535,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,NaN
3,188536,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,NaN
4,188537,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes


## EDA & DATA PREPROCESSING

In [5]:
# DF infomation:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     183450 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      186081 non-null  object
 11  clean_title   167114 non-null  object
 12  price         188533 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.7+ MB


In [6]:
# Descriptive statistics of target variable:
np.round(df['price'].describe().T, 3)

count     188533.000
mean       43878.016
std        78819.522
min         2000.000
25%        17000.000
50%        30825.000
75%        49900.000
max      2954083.000
Name: price, dtype: float64

In [7]:
# Null values as percentage of total data:
(df.isnull().sum().sort_values(ascending=False) / len(df)) * 100

clean_title     11.360876
fuel_type        2.696080
accident         1.300568
id               0.000000
brand            0.000000
model            0.000000
model_year       0.000000
milage           0.000000
engine           0.000000
transmission     0.000000
ext_col          0.000000
int_col          0.000000
price            0.000000
dtype: float64

In [8]:
# Investigation of "clean_title":
df['clean_title'].value_counts()

clean_title
Yes    167114
Name: count, dtype: int64

In [9]:
# Clean title contains only "Yes" for the most of the values.
# We will assume that the missing values are "No".
df['clean_title'] = df['clean_title'].fillna(value='No')
df['clean_title'].value_counts()

clean_title
Yes    167114
No      21419
Name: count, dtype: int64

In [10]:
# Investigate 'fuel_type'
df['fuel_type'].value_counts()

fuel_type
Gasoline          165940
Hybrid              6832
E85 Flex Fuel       5406
Diesel              3955
–                    781
Plug-In Hybrid       521
not supported         15
Name: count, dtype: int64

In [11]:
# Fill NA for fuel type with "–":
df['fuel_type'] = df['fuel_type'].fillna(value='–')  # This value was already present in the "fuel_type" column
df['fuel_type'].value_counts()

fuel_type
Gasoline          165940
Hybrid              6832
–                   5864
E85 Flex Fuel       5406
Diesel              3955
Plug-In Hybrid       521
not supported         15
Name: count, dtype: int64

In [12]:
# Extract pure electric fuel type from engine:
electric_fuel_type_index =\
df[(df['fuel_type'] == '–') &
    (df['engine'].str.contains(pat='Electric'))].index

# Update matching index to electric:
for idx in electric_fuel_type_index:
    df.at[idx, 'fuel_type'] = 'Electric'

In [20]:
cond1 = 'brand == "Tesla" & '
cond2 = 'model == "Model 3 Long Range" & '
cond3 = 'model_year == 2020'
full_cond = cond1 + cond2 + cond3
df.query(expr=full_cond)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
51,51,Tesla,Model 3 Long Range,2020,38360,–,Dual Motor - Standard,Automatic,White,Black,None reported,No,59598
7807,7807,Tesla,Model 3 Long Range,2020,30455,–,Dual Motor - Standard,Automatic,Silver,Black,None reported,No,39798
8064,8064,Tesla,Model 3 Long Range,2020,42094,Electric,271.0HP Electric Motor Electric Fuel System,A/T,White,White,At least 1 accident or damage reported,Yes,32000
13012,13012,Tesla,Model 3 Long Range,2020,45843,Electric,Electric,1-Speed Automatic,Black,Black,None reported,Yes,20784
19218,19218,Tesla,Model 3 Long Range,2020,29336,Gasoline,Electric,1-Speed Automatic,Red Multi,Black,None reported,Yes,34995
19486,19486,Tesla,Model 3 Long Range,2020,24906,–,Dual Motor - Standard,Automatic,Gray,Black,None reported,No,143998
20087,20087,Tesla,Model 3 Long Range,2020,59500,Electric,425.0HP Electric Motor Electric Fuel System,1-Speed A/T,Black,Black,None reported,Yes,112500
21263,21263,Tesla,Model 3 Long Range,2020,1877,–,Dual Motor - Standard,Automatic,Black,Black,None reported,No,40998
22356,22356,Tesla,Model 3 Long Range,2020,19800,Electric,425.0HP Electric Motor Electric Fuel System,A/T,White,Black,None reported,Yes,46999
23957,23957,Tesla,Model 3 Long Range,2020,23749,Electric,Electric,1-Speed Automatic,White,Black,None reported,No,45855


In [23]:
df[
    (df['engine'].str.contains('Electric')) &
    (df['fuel_type'] == 'Gasoline')
]

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
44,44,Rivian,R1S Adventure Package,2023,7000,Gasoline,835.0HP Electric Motor Electric Fuel System,8-Speed A/T,Green,Black,None reported,Yes,145000
194,194,Tesla,Model 3 Standard Range Plus,2021,42094,Gasoline,Electric Motor Electric Fuel System,A/T,Black,Black,None reported,Yes,32750
211,211,Tesla,Model Y Long Range,2020,60000,Gasoline,425.0HP Electric Motor Electric Fuel System,A/T,White,White,At least 1 accident or damage reported,Yes,60000
846,846,Tesla,Model 3 Long Range,2021,27123,Gasoline,271.0HP Electric Motor Electric Fuel System,1-Speed A/T,White,Black,None reported,Yes,33333
1062,1062,Tesla,Model S 100D,2018,11806,Gasoline,518.0HP Electric Motor Electric Fuel System,A/T,White,Black,At least 1 accident or damage reported,Yes,46000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187530,187530,Tesla,Model Y Performance,2018,78101,Gasoline,455.0HP Electric Motor Electric Fuel System,A/T,White,Black,None reported,Yes,41899
188009,188009,Toyota,Sequoia Platinum,2022,2894,Gasoline,437.0HP 3.4L V6 Cylinder Engine Gas/Electric H...,10-Speed A/T,Green,Gray,None reported,Yes,74500
188216,188216,Tesla,Model X 75D,2016,103000,Gasoline,518.0HP Electric Motor Electric Fuel System,A/T,Black,Black,None reported,Yes,17500
188248,188248,Rivian,R1S Launch Edition,2022,2525,Gasoline,835.0HP Electric Motor Electric Fuel System,1-Speed A/T,Orange,White,None reported,Yes,81500


In [13]:
df[df['fuel_type'] == '–']

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
51,51,Tesla,Model 3 Long Range,2020,38360,–,Dual Motor - Standard,Automatic,White,Black,None reported,No,59598
532,532,Dodge,Challenger R/T Scat Pack,2021,35000,–,–,8-Speed A/T,Blue,Gray,None reported,Yes,48000
1155,1155,Ford,Mustang GT Premium,2017,143600,–,–,6-Speed M/T,White,Black,At least 1 accident or damage reported,Yes,31000
1307,1307,Toyota,Land Cruiser Base,1994,138033,–,–,A/T,Green,Beige,None reported,Yes,11999
1362,1362,Ford,Mustang EcoBoost Premium,2016,43000,–,–,A/T,Blue,Black,None reported,Yes,15500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187495,187495,Nissan,Pathfinder SL,2014,87305,–,284.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,Brown,Gray,At least 1 accident or damage reported,Yes,10000
187664,187664,Mercedes-Benz,SLS AMG Base,1993,14586,–,–,A/T,Red,Silver,None reported,Yes,65000
187820,187820,Chrysler,Pacifica Touring,2017,57300,–,–,9-Speed A/T,Blue,Black,None reported,Yes,31700
188189,188189,Chrysler,300 Touring,1994,79785,–,–,A/T,Green,Beige,At least 1 accident or damage reported,Yes,9499


In [14]:
# Investigation of "accident":
df['accident'].value_counts()

accident
None reported                             144514
At least 1 accident or damage reported     41567
Name: count, dtype: int64

In [15]:
# Fill null values with "None reported"
df['accident'] = df['accident'].fillna(value='None reported')
df['accident'].value_counts()

accident
None reported                             146966
At least 1 accident or damage reported     41567
Name: count, dtype: int64

In [16]:
# Replace "–" with None for all values:
df = df.replace(to_replace='–', value='None')  # for the sake of readablility.

In [17]:
# Investigate Transmission:
df['transmission'].value_counts()

transmission
A/T                                                   49904
8-Speed A/T                                           20645
Transmission w/Dual Shift Mode                        19255
6-Speed A/T                                           18044
6-Speed M/T                                           11998
7-Speed A/T                                           11124
Automatic                                             10691
8-Speed Automatic                                      8431
10-Speed A/T                                           8044
9-Speed A/T                                            3866
5-Speed A/T                                            3217
10-Speed Automatic                                     3164
6-Speed Automatic                                      2799
4-Speed A/T                                            2546
5-Speed M/T                                            2409
9-Speed Automatic                                      2325
CVT Transmission           

In [18]:
# Transmission values exhibit certain redundant data. We will define a function to tackle this issue:
def fix_transmission(transmission_value: str) -> str:
    """Fix transmission value and remove redundant data"""
    
    # Standardize Automatic Transmissions (A/T)
    automatic_keywords = ['A/T', 'Automatic', 'Auto', 'AT']
    if any(keyword in transmission_value for keyword in automatic_keywords):
        if '8-Speed' in transmission_value:
            return '8-Speed Automatic'
        elif '10-Speed' in transmission_value:
            return '10-Speed Automatic'
        elif '9-Speed' in transmission_value:
            return '9-Speed Automatic'
        elif '7-Speed' in transmission_value:
            return '7-Speed Automatic'
        elif '6-Speed' in transmission_value:
            return '6-Speed Automatic'
        elif '5-Speed' in transmission_value:
            return '5-Speed Automatic'
        elif '4-Speed' in transmission_value:
            return '4-Speed Automatic'
        elif '2-Speed' in transmission_value:
            return '2-Speed Automatic'
        elif '1-Speed' in transmission_value:
            return '1-Speed Automatic'
        else:
            return 'Automatic'
    
    # Standardize Manual Transmissions (M/T)
    manual_keywords = ['M/T', 'Manual', 'Mt']
    if any(keyword in transmission_value for keyword in manual_keywords):
        if '8-Speed' in transmission_value:
            return '8-Speed Manual'
        elif '7-Speed' in transmission_value:
            return '7-Speed Manual'
        elif '6-Speed' in transmission_value:
            return '6-Speed Manual'
        elif '5-Speed' in transmission_value:
            return '5-Speed Manual'
        else:
            return 'Manual'
    
    # Standardize CVT (Continuously Variable Transmission)
    if 'CVT' in transmission_value:
        return 'CVT'

    # Handle special cases
    if 'Single-Speed Fixed Gear' in transmission_value:
        return 'Single-Speed Fixed Gear'
    if 'Transmission w/Dual Shift Mode' in transmission_value:
        return 'Dual Shift Mode'
    if 'Transmission Overdrive Switch' in transmission_value:
        return 'Transmission Overdrive Switch'
    
    # Handle undefined or special values
    if transmission_value in ['None', 'Variable', 'F', '2']:
        return 'Other'
    
    # Default case: return the original value if no match
    return transmission_value

In [ ]:
# Fix transmission column:
df['transmission'] = np.vectorize(fix_transmission)(df['transmission'])
df['transmission'].value_counts()

# FEATURE ENGINEERING

In [ ]:
# One-hot-encode:
pd.get_dummies(df['accident'],
              prefix="",
              prefix_sep='')

In [ ]:
# Further investigate engine column:
df['engine'].value_counts()